In [ ]:
import os

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
try:
    import open3d as o3d
except ImportError:
    import sys
    print(sys.exc_info())
import pandas as pd
from scipy import interpolate

from dosipy.utils.dataloader import load_ear_data
from dosipy.utils.integrate import elementwise_quad, elementwise_dblquad
from dosipy.utils.viz import (set_colorblind, scatter_2d, scatter_3d,
                              fig_config)
from helpers import (clean_df, export_pcd, export_fields,
                     poynting_vector, get_imcolors, export_rect_idx)

In [ ]:
polarization = 'tm'
f = 60

# Spatial-average APD

In [ ]:
# load surface data

df = load_ear_data(polarization, f, surface='front')
df = clean_df(df)
xyz = export_pcd(df)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
center = pcd.get_center()
# cframe = o3d.geometry.TriangleMesh.create_coordinate_frame(
#     size=9, origin=center+np.array([6, -25, -20])
# )
# o3d.visualization.draw_geometries([pcd, cframe])

In [ ]:
# postprocessing on the surface of the model

E, H = export_fields(df)
Sx, Sy, Sz = poynting_vector(E, H)

pcd.estimate_normals()
n = np.asarray(pcd.normals)

APD = np.abs(Sx.real * n[:, 0] + Sy.real * n[:, 1] + Sz.real * n[:, 2])

In [ ]:
set_colorblind()
fig_config(latex=True, scaler=2, text_size=18)
label = '$APD$ [W/m$^2$]'
fig, ax = scatter_3d({'$z$ [mm]': xyz[:, 2],
                      '$x$ [mm]': xyz[:, 0],
                      '$y$ [mm]': xyz[:, 1],
                      label: APD},
                     elev=[10], azim=[105])

In [ ]:
# extract points visible from the plane wave incidence POV (-x direction)

diameter = np.linalg.norm(
    pcd.get_max_bound() - pcd.get_min_bound()
)
radius = 10 ** 5
camera = [center[0] + diameter, center[1], center[2]]
_, mask = pcd.hidden_point_removal(camera, radius)
xyz_zy = xyz[mask]
APD_zy = APD[mask]
n_zy = n[mask]

In [ ]:
# find appropriate domain

pAPD_idx = np.where(APD_zy == APD_zy.max())[0][0]
avg_center = xyz_zy[pAPD_idx, 2], xyz_zy[pAPD_idx, 1]
avg_center = [avg_center[0], avg_center[1]]
edge_length = 10
area = edge_length ** 2
origin, _ = export_rect_idx(xyz=xyz_zy,
                            center=[avg_center[0], avg_center[1]],
                            edge_length=edge_length,
                            view='zy')
set_colorblind()
fig_config(latex=True, scaler=1.5, text_size=16)
fig, ax = scatter_2d({'$z$ [mm]': xyz_zy[:, 2],
                      '$y$ [mm]': xyz_zy[:, 1],
                      label: APD_zy}, s=0.1, figsize=(2.950 * 1.5, 2.950 * 1.5))
patch_rect = Rectangle(origin, edge_length, edge_length, fc='None', lw=2)
ax.add_patch(patch_rect)
ax.invert_xaxis()

In [ ]:
# find the max APDav and the corresponding averaging surface

pAPD_idx = np.where(APD_zy == APD_zy.max())[0][0]
avg_center = xyz_zy[pAPD_idx, 2], xyz_zy[pAPD_idx, 1]
z_left, z_right, y_left, y_right = (0, 0, 0, 0) 
av_dict = {'origin': [], 'xyz': [], 'APDav': []}

for zc in np.linspace(avg_center[0] - z_left, avg_center[0] + z_right, 21):
    for yc in np.linspace(avg_center[1]- y_left, avg_center[1] + y_right, 21):
        _origin, _idx_rect = export_rect_idx(xyz=xyz_zy,
                                             center=[zc, yc],
                                             edge_length=edge_length,
                                             view='zy')
        _xyz_rect = xyz_zy[_idx_rect]
        _APD_rect = APD_zy[_idx_rect]
        _n_rect = n_zy[_idx_rect]
        _APDav = elementwise_dblquad(points=np.c_[_xyz_rect[:, 2], _xyz_rect[:, 1]],
                                     values=_APD_rect,
                                     degree=31,
                                     interp_func=interpolate.LinearNDInterpolator, fill_value=0
                                    ) / area
        av_dict['origin'].append(_origin)
        av_dict['xyz'].append(_xyz_rect)
        av_dict['APDav'].append(_APDav)
        
pAPDav_idx_rect = np.argmax(av_dict['APDav'])
pAPDav_origin = av_dict['origin'][pAPDav_idx_rect]
pAPDav_xyz = av_dict['xyz'][pAPDav_idx_rect]
pAPDav = av_dict['APDav'][pAPDav_idx_rect]

In [ ]:
set_colorblind()
fig_config(latex=True, scaler=1.5, text_size=16)
fig, ax = scatter_2d({'$z$ [mm]': xyz_zy[:, 2],
                      '$y$ [mm]': xyz_zy[:, 1],
                      label: APD_zy}, s=0.1, figsize=(2.950 * 1.5, 2.950 * 1.5))
ax.set_title(f'$pAPD_{{av}} = {pAPDav:.2f}$ W/m$^2$')
patch_rect = Rectangle(pAPDav_origin, edge_length, edge_length, fc='None', lw=2)
ax.add_patch(patch_rect)
ax.invert_xaxis()
print(pAPDav_origin)

In [ ]:
# apply this only for f = 60 GHz after pAPDav is already evaluated on 4cm^2 square area

if f == 60:
    def reduction():
        _pAPDav_center = [pAPDav_origin[0] + edge_length / 2,
                          pAPDav_origin[1] + edge_length / 2]
        _origin, _idx_rect = export_rect_idx(xyz=xyz_zy,
                                             center=_pAPDav_center,
                                             edge_length=edge_length / 2,
                                             view='zy')
        _xyz_rect = xyz_zy[_idx_rect]
        _APD_rect = APD_zy[_idx_rect]
        _APDav = elementwise_dblquad(points=np.c_[_xyz_rect[:, 2], _xyz_rect[:, 1]],
                                     values=_APD_rect,
                                     degree=11,
                                     interp_func=interpolate.LinearNDInterpolator, fill_value=0) / (area / 4)
        return _origin, _xyz_rect, _APDav

    pAPDav_origin, pAPDav_xyz, pAPDav_reduction = reduction()
    pAPDav_reduction
    
    set_colorblind()
    fig_config(latex=True, scaler=1.5, text_size=16)
    fig, ax = scatter_2d({'$z$ [mm]': xyz_zy[:, 2],
                          '$y$ [mm]': xyz_zy[:, 1],
                          label: APD_zy}, s=0.1, figsize=(2.950 * 1.5, 2.950 * 1.5))
    ax.set_title(f'$pAPD_{{av}} = {pAPDav_reduction:.2f}$ W/m$^2$')
    patch_rect = Rectangle(pAPDav_origin, edge_length / 2, edge_length / 2, fc='None', lw=2)
    ax.add_patch(patch_rect)
    ax.invert_xaxis()

# Spatial-average TPD

In [ ]:
# load the volume data for the block of tissue where the front face is the region that yields max APDav

fname_block = os.path.join('data', f'loss_3D_ear_{f}GHz_{polarization.upper()}_front.txt')
df_block = (pd.read_csv(fname_block, names=['x [mm]', 'y [mm]', 'z [mm]', 'Pl [W/m^3]'],
                        header=None, delim_whitespace=True, skiprows=[0, 1])
            [lambda x: (x['Pl [W/m^3]'] != 0.0)
                       & (x['y [mm]'] >= pAPDav_xyz[:, 1].min())
                       & (x['y [mm]'] <= pAPDav_xyz[:, 1].max())
                       & (x['z [mm]'] >= pAPDav_xyz[:, 2].min())
                       & (x['z [mm]'] <= pAPDav_xyz[:, 2].max())])
df_block.reset_index(drop=True, inplace=True)

xyz_block = export_pcd(df_block)
Pl_block = df_block['Pl [W/m^3]'].to_numpy()
tissue_density = 1049

In [ ]:
set_colorblind()
fig_config(latex=True, scaler=2, text_size=18)
label = '$SAR$ [W/kg]'
fig, ax = scatter_3d({'$z$ [mm]': xyz_block[:, 2],
                      '$x$ [mm]': xyz_block[:, 0],
                      '$y$ [mm]': xyz_block[:, 1],
                      label: Pl_block / tissue_density},
                      elev=[15], azim=[105])

In [ ]:
# compute the TPD distribution by integrating SAR depth-wise (-x direction)

df_rect = df_block.groupby(['y [mm]', 'z [mm]']).apply(
    lambda col: pd.Series({'TPD [W/m^2]': elementwise_quad(col['x [mm]'].to_numpy() / 1000,  # convert to meters
                                                           col['Pl [W/m^3]'].to_numpy(),
                                                           degree=11)})
    ).reset_index()

In [ ]:
set_colorblind()
fig_config(latex=True, scaler=1.5, text_size=16)
label = '$TPD$ [W/m$^2$]'
fig, ax = scatter_2d({'$z$ [mm]': df_rect['z [mm]'].to_numpy(),
                      '$y$ [mm]': df_rect['y [mm]'].to_numpy(),
                      label: df_rect['TPD [W/m^2]'].to_numpy()}, s=3,
                     figsize=(2.950 * 1.915, 2.950 * 1.5))
ax.invert_xaxis()

In [ ]:
ds = 1
TPD_av = elementwise_dblquad(points=df_rect[['z [mm]', 'y [mm]']].to_numpy()[::ds],
                             values=df_rect['TPD [W/m^2]'].to_numpy()[::ds],
                             degree=31,
                             interp_func=interpolate.LinearNDInterpolator, fill_value=0
                            ) / area
TPD_av